# บทเรียนที่ 1: scraping เบื้องต้น

## ขั้นตอนการ scrape โดยทั่วไป
1. หาข้อมูล ว่าอยู่ตรงไหน และจะดึงมาได้อย่างไร
2. ดึงข้อมูลมาเป็น text (HTTP requests)
3. ใช้ regular expression หรือ BeautifulSoup ในการแยกข้อมูลที่สนใจออกมา
4. สร้าง loop ครอบข้อ 2-3

## ข้อมูลโรงพยาบาลรัฐ
ข้อมูลโรงพยาบาลทั่วประเทศ ในเว็บ สธ. http://gishealth.moph.go.th/healthmap/gmap.php โดยมีหน้ารายละเอียดต่าง ๆ ในระดับ รพ. ดังนี้
* ข้อมูลทั่วไป ที่ตั้ง ผู้บริหาร http://gishealth.moph.go.th/healthmap/info.php
* ข้อมูลบุคลากร http://gishealth.moph.go.th/healthmap/infopersonal.php
* ข้อมูลแพทย์เฉพาะทาง http://gishealth.moph.go.th/healthmap/infospecialty.php
* ข้อมูลสถานที่และเครื่องมือแพทย์ http://gishealth.moph.go.th/healthmap/infoequip.php
* ข้อมูลประเภทบริการ http://gishealth.moph.go.th/healthmap/infotype.php
* ข้อมูลการให้บริการ http://gishealth.moph.go.th/healthmap/infoservice.php

โดยการจะเข้าถึงข้อมูล ต้องมีรหัสโรงพยาบาล 5 หลัก ใส่เป็น maincode เช่น http://gishealth.moph.go.th/healthmap/info.php?maincode=10667

## เริ่มจากการทำ HTTP request ก่อน

In [60]:
import requests
r = requests.get('http://gishealth.moph.go.th/healthmap/info.php?maincode=10668')
print(r.status_code)
# 200 = successful

200


In [61]:
r.text
# see what we get in r.text

'\ufeff<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n\t<head>\r\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>\r\n<script type="text/javascript">\r\n  var _gaq = _gaq || [];\r\n  _gaq.push([\'_setAccount\', \'UA-18733385-1\']);\r\n  _gaq.push([\'_trackPageview\']);\r\n  (function() {\r\n    var ga = document.createElement(\'script\'); ga.type = \'text/javascript\'; ga.async = true;\r\n    ga.src = (\'https:\' == document.location.protocol ? \'https://ssl\' : \'http://www\') + \'.google-analytics.com/ga.js\';\r\n    var s = document.getElementsByTagName(\'script\')[0]; s.parentNode.insertBefore(ga, s);\r\n  })();\r\n</script>\r\n    <title>ระบบสารสนเทศภฺมิศาสตร์ทรัพยากรสุขภาพ  โรงพยาบาลสุรินทร์</title>\r\n    <script type="text/javascript" src="jquery/jquery-1.3.2.min.js"></script> \r\n\t<script type="text/javascript" src="jquery/jquery.easing.1.3.

## ลองใช้ regular expression
เข้าไปลองที่นี่ได้เลย https://regex101.com/#python แปะ source code (r.text) เข้าไป สมมติว่าเราต้องการตัวแปรเหล่านี้จากเพจ info.php
* ชื่อโรงพยาบาล
* ที่อยู่
* เบอร์โทรศัพท์

(ดูเฉลยได้ที่ https://regex101.com/r/RZz7W7/2)

In [62]:
import re
regex = '<td width="70%"  class="tdbold">\s+<div  >\s*([^<&]+).*ที่อยู่\s+([^<&]+).*โทร.\s*([0-9๐-๙- ,]+)'

In [63]:
# using group...
m = re.search(regex, r.text, re.MULTILINE | re.DOTALL)
[m.group(1), m.group(2), m.group(3)]

['โรงพยาบาลสุรินทร์',
 'เลขที่ 68 ถ.หลักเมือง ต.ในเมือง อ.เมือง จ.สุรินทร์ 32000          ',
 '044-511757']

In [64]:
re.findall(regex, r.text, re.MULTILINE | re.DOTALL)

[('โรงพยาบาลสุรินทร์',
  'เลขที่ 68 ถ.หลักเมือง ต.ในเมือง อ.เมือง จ.สุรินทร์ 32000          ',
  '044-511757')]

## ลองสร้าง loop ทั้งหมด
ในเมื่อเราไม่รู้ว่ารหัส 5 หลักของ รพ. คืออะไรบ้าง เราก็ loop โลด

In [65]:
# test zfill function
tmp = 89
str(tmp).zfill(5)

'00089'

In [77]:
regex = '<td width="70%"  class="tdbold">\s+<div  >\s*([^<&]+).*ที่อยู่\s+([^<&]+).*โทร.\s*([0-9๐-๙- ,]+)'
for i in range(1, 6):
    r = requests.get('http://gishealth.moph.go.th/healthmap/info.php?maincode=' + str(i).zfill(5))
    print(re.findall(regex, r.text, re.MULTILINE | re.DOTALL))

[('สำนักงานสาธารณสุขจังหวัดสมุทรปราการ', '19 ซอย 35 อัศวนนท์ 2 ถ.สุขุมวิท ต.ปากน้ำ อ.เมือง จ.สมุทรปราการ 10270          ', '02-3895980')]
[('สำนักงานสาธารณสุขจังหวัดนนทบุรี', 'ศูนย์ราชการจังหวัดนนทบุรี ม.8 ถ.รัตนาธิเบศร์ ต.บางกระสอ อ.เมือง จ.นนทบุรี  11000          ', '02 9503071-6')]
[('สำนักงานสาธารณสุขจังหวัดปทุมธานี', '๑๔ ถนนรัฐอำนวย ตำบลบางปรอก อำเภอเมืองปทุมธานี          ', '๐ ๒๕๘๑ ๖๔๕๔ ')]
[('สำนักงานสาธารณสุขจังหวัดพระนครศรีอยุธยา', 'ค 130 ถนนอู่ทอง ต.หอรัตนไชย          ', '035-241520')]
[('สำนักงานสาธารณสุขจังหวัดอ่างทอง', '122 ม.4 ถ.โพธิ์พระยา-ท่าเรือ  ต.บ้านอิฐ อ.เมือง จ.อ่างทอง          ', '035-611222 ')]


## Putting it all together

In [110]:
import requests
import re
import csv

with open('hospital.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
    csvwriter = csv.writer(csvfile, quoting=csv.QUOTE_ALL)
    regex = '<td width="70%"  class="tdbold">\s+<div  >\s*([^<&]+).*ที่อยู่\s+([^<&]+).*โทร.\s*([0-9๐-๙- ,]+)'
    for i in range(1, 6):
        r = requests.get('http://gishealth.moph.go.th/healthmap/info.php?maincode=' + str(i).zfill(5))
        m = re.search(regex, r.text, re.MULTILINE | re.DOTALL)
        #m = re.findall(regex, r.text, re.MULTILINE | re.DOTALL)
        csvwriter.writerow([str(i).zfill(5), m.group(1).strip(), m.group(2).strip(), m.group(3).strip()])

ถึงตอนนี้ เราก็แค่ให้ loop รันตั้งแต่ 1 ถึง 99,999 ก็เสร็จแล้ว! แต่กว่าจะได้ยังงั้นก็อาจจะแก่ตายก่อน คอขวดหลักที่ทำให้ช้า มีอยู่สองที่
1. ตอน requests.get ต้องรอ response จาก server
2. การเขียนไฟล์หลาย ๆ รอบ

อันที่ 2 เป็นปัญหาไม่ใหญ่มาก ไว้ค่อยแก้ทีหลังละกัน (แป่ว)... อันที่ 1 เป็นปัญหาหลักของเรา ซึ่งแก้ได้ด้วยการ thread หรือพูดง่าย ๆ ว่า run script หลาย ๆ รอบนั่นเอง เช่น thread แรก ให้หา รพ. ตั้งแต่เลข 1 ถึง 10,000 ฯลฯ ซึ่งจะเขียน `hospital_1.py`, `hospital_2.py`, ... ก็ได้ หรือถ้าขี้เกียจ ก็น่าจะเขียนให้ script รับ parameter ได้ ว่าจะรันตั้งแต่เท่าไหร่ถึงเท่าไหร่ (แล้วก็ต้องบอก output file ด้วยเนอะ ไม่งั้นก็เขียนทับกัน)

เปิดไฟล์ใหม่ code ตามด้านล่าง ก็เขียน
```
$ python hospital.py 1 100 out_1.csv
```
ได้เลย

```python
import requests
import re
import csv
import sys

with open(sys.argv[3], 'w', newline='', encoding='utf-8-sig') as csvfile:
    csvwriter = csv.writer(csvfile, quoting=csv.QUOTE_ALL)
    regex = '<td width="70%"  class="tdbold">\s+<div  >\s*([^<&]+).*ที่อยู่\s+([^<&]+).*โทร.\s*([0-9๐-๙- ,]+)'
    for i in range(int(sys.argv[1]), int(sys.argv[2]) + 1):
        r = requests.get('http://gishealth.moph.go.th/healthmap/info.php?maincode=' + str(i).zfill(5))
        m = re.search(regex, r.text, re.MULTILINE | re.DOTALL)
        #m = re.findall(regex, r.text, re.MULTILINE | re.DOTALL)
        csvwriter.writerow([str(i).zfill(5), m.group(1).strip(), m.group(2).strip(), m.group(3).strip()])
```

จบบทเรียนที่ 1 แล้ววววว!!!